In [ ]:
import numpy as np
import pandas as pd
from sklearn.calibration import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import SequentialFeatureSelector
import seaborn as sns
from sklearn.feature_selection import SequentialFeatureSelector
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import statsmodels.api as sm


In [ ]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')
#df2

In [ ]:
df1 = df1.drop(columns=['sub_area'])
df2 = df2.drop(columns=['sub_area','row ID'])

In [ ]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)

In [ ]:
categorical_columns = df1.select_dtypes(include=['object']).columns

# Label encode categorical columns
label_encoder = LabelEncoder()
for col in categorical_columns:
   df1[col] = label_encoder.fit_transform(df1[col])
   df2[col] = label_encoder.transform(df2[col])

In [ ]:
X = df_onehot.drop(columns=['price_doc',],axis=1)
y = df_onehot['price_doc']

In [ ]:
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
df2_encoded = imputer.fit_transform(test_onehot)

In [ ]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_testscaled = scaler.fit_transform(df2_encoded)

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
df2_encoded_scaled = scaler.fit_transform(df2_encoded)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

variance_threshold = 0.01 # Set your desired threshold
selector = VarianceThreshold(threshold=variance_threshold)
X_train_high_variance = selector.fit_transform(X_scaled)
X_test_high_variance = selector.transform(df2_encoded_scaled)

In [ ]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_scaled)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(X_testscaled)

In [ ]:
import statsmodels.api as sm
import pandas as pd

# Convert scaled_df1 to a DataFrame without specifying columns

# Add a constant term to the feature matrix
X_with_const = sm.add_constant(X_scaled)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.05

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)

# Select columns in the DataFrame
X_selected = X_scaled[selected_features]


In [ ]:
#keep those columns in X_testscaled that are in X_selected
X_testscaled = X_testscaled[X_selected.columns]

In [ ]:
X_selected.shape

In [ ]:
X_testscaled.shape

In [ ]:
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_selected)
X_test_pca = pca.transform(X_testscaled)

In [ ]:
poly_features=PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
X_train_poly = poly_features.fit_transform(X_train_pca)
X_test_poly = poly_features.transform(X_test_pca)

In [ ]:
# Create and train the Ridge regression model
alpha_value = 0.5  # You can adjust the alpha parameter based on your needs
ridge_reg = Ridge(alpha=alpha_value, random_state=42)
ridge_reg.fit(X3, y)

# Print the coefficients and intercept
print("Coefficients:", ridge_reg.coef_)
print("Intercept:", ridge_reg.intercept_)

used.append('Ridge Regression Alpha = 0.5')

# Make predictions on the test data
y_pred_ridge = ridge_reg.predict(test3)

In [ ]:
additional_data = pd.read_csv("test.csv")

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': md_probs  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions42.csv', index=False)